# Poisson Regression with Random Effects for Over-dispersion
## COVID-19 Cases Analysis in Lima Districts

**Author:** [Your Name]  
**Course:** Computational Statistics  
**Date:** August 2024

This notebook implements Poisson regression models with random effects to handle over-dispersion in COVID-19 case data from Lima districts.


In [ ]:
# Load required packages
using CSV
using DataFrames
using Distributions
using Random
using LinearAlgebra
using Optim
using StatsBase
using StatsFuns
using SpecialFunctions
using QuadGK
import CairoMakie as MK
using Printf

Random.seed!(123)  # For reproducibility


## 1. Data Loading and Exploration


In [ ]:
# Load the COVID-19 data
covid_data = CSV.read("course-computational-statistics-julia-main/data/simulated/01-lima-over-dispersion.csv", DataFrame)

# Display basic information
println("Dataset dimensions: ", size(covid_data))
println("\nFirst few rows:")
first(covid_data, 5)


In [ ]:
# Classical Poisson log-likelihood function
function loglik_poisson(β, data)
    y = data.cases
    x = data.predictor
    N = data.population
    
    η = β[1] .+ x * β[2]
    λ = N .* exp.(η)
    
    loglik = -sum(λ) + sum(y .* η)
    return loglik
end

# Fit classical Poisson regression
function fit_poisson(data)
    β0 = [0.0, 0.0]
    negloglik(β) = -loglik_poisson(β, data)
    
    result = optimize(negloglik, β0, Newton(); autodiff = :forward)
    β_hat = Optim.minimizer(result)
    
    return β_hat, result
end

# Fit the model
β_poisson, opt_result = fit_poisson(covid_data)
println("Classical Poisson regression coefficients:")
println("β₀ (intercept): ", round(β_poisson[1], digits=4))
println("β₁ (predictor): ", round(β_poisson[2], digits=4))
println("Exp(β₁) (rate ratio): ", round(exp(β_poisson[2]), digits=4))


In [ ]:
# Monte Carlo test for overdispersion
function overdispersion_test(data, β, n_sim=1000)
    λ_fitted = data.population .* exp.(β[1] .+ data.predictor * β[2])
    observed_stat = sum((data.cases - λ_fitted).^2 ./ λ_fitted)
    
    sim_stats = zeros(n_sim)
    for i in 1:n_sim
        y_sim = rand.(Poisson.(λ_fitted))
        sim_stats[i] = sum((y_sim - λ_fitted).^2 ./ λ_fitted)
    end
    
    p_value = mean(sim_stats .> observed_stat)
    return observed_stat, sim_stats, p_value
end

obs_stat, sim_stats, p_val = overdispersion_test(covid_data, β_poisson, 2000)
println("Monte Carlo Overdispersion Test Results:")
println("Observed statistic: ", round(obs_stat, digits=2))
println("P-value: ", round(p_val, digits=4))
